In [154]:
import pandas as pd

df = pd.read_csv('resources/train.csv')

# Exploratory Data Analysis

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

There are 81 columns in the dataset and the dataset contains 1460 rows. The label is SalePrice

There's no doublon in the dataset but a lot of NA values.

The columns with many missing values are: PoolQC(NA => No Pool), Fence(NA => No Fence), MiscFeature, MasVnrType, Alley (around 50% of missing values)

The columns that seems to be useless are: Id, YrSold, PoolArea, Utilities, LandContour, Street, LandSlope, LotConfig(pas sûr)

The columns that seems useful are: MSSubClass, MSZoning, LotFrontage, LotArea, LotShape, LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, MoSold, SaleType, SaleCondition

In [156]:
# Copy the dataset without the useless columns
columns_to_drop = ['Id', 'PoolQC', 'Fence', 'MiscFeature', 'YrSold', 'PoolArea', 'Alley', 'Utilities', 'LandContour', 'Street']

df = df.drop(columns=columns_to_drop)

# Data Preprocessing and encoding

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import Lasso

# Initialize the encoders
one_hot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()

# Set X and y
X = df.drop(columns='SalePrice')
y = df['SalePrice']

# One hot encoding
X = one_hot_encoder.fit_transform(X)

# Label encoding
y = label_encoder.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model

In [158]:
# Initialize the model
model = Lasso()

# Train the model
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
model.score(X_test, y_test)

0.879469486662082